# Load dataset

In [129]:
import warnings
warnings.filterwarnings("ignore")

In [130]:
from sentence_transformers import SentenceTransformer
import pandas as pd

In [131]:
vietnamese_history_dataset_path = "../../datasets/vietnamese_history_dataset/data.json"
df = pd.read_json(vietnamese_history_dataset_path)
df.head()

,title,content,type
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title


In [132]:
df["content_embedding"] = None
df["content_embeded"] = None

for idx in range(len(df)):
    df["content_embedding"][idx] = df["title"][idx] + " " + df["content"][idx]

df.head()

,title,content,type,content_embedding,content_embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter,BUỔI ĐẦU LỊCH SỬ NƯỚC TA Chương: BUỔI ĐẦU LỊCH...,None
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA Chương: BUỔI...,None
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title,Những dấu tích của Người tối cổ được tìm thấy ...,None
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",None
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title,Giai đoạn phát triển của Người tinh khôn có gì...,None


In [133]:
import glob
glob.glob("../../embedding_models/*")

['../../embedding_models\\all-MiniLM-L6-v2',
 '../../embedding_models\\bert-base-vietnamese-uncased',
 '../../embedding_models\\distiluse-base-multilingual-cased-v2',
 '../../embedding_models\\multilingual-e5-small',
 '../../embedding_models\\phobert-base-v2',
 '../../embedding_models\\phobert-large',
 '../../embedding_models\\README.md',
 '../../embedding_models\\sup-SimCSE-VietNamese-phobert-base',
 '../../embedding_models\\vietnamese-bi-encoder',
 '../../embedding_models\\vietnamese-embedding']

In [134]:
special_characters = "❉ ♡ƪ(ˆ◡ˆ)ʃ♪ 卍 ₯"
sentence = df["content"][0] + " " + special_characters
print(sentence)

Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA ❉ ♡ƪ(ˆ◡ˆ)ʃ♪ 卍 ₯


# Các chỉ số đánh giá hiệu quả tokenizer của các model embedding

| Chỉ số                            | Mô tả                                                                                                     | Cách tính                                                                                                                                                                                  |
| --------------------------------- | --------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **1. avg\_tokens\_per\_sample**   | Số **token trung bình** trên mỗi đoạn văn bản (sample)                                                    | Tính trung bình số token: <br> $\text{Avg} = \frac{1}{N} \sum_{i=1}^{N} T_i$ <br> với $T_i$ là số token của văn bản thứ i                                                                  |
| **2. std\_tokens\_per\_sample**   | **Độ lệch chuẩn** số token giữa các văn bản                                                               | Dùng công thức độ lệch chuẩn: <br> $\sigma = \sqrt{ \frac{1}{N} \sum_{i=1}^{N} (T_i - \text{Avg})^2 }$                                                                                     |
| **3. max\_tokens**                | Số lượng **token lớn nhất** trong 1 đoạn văn bản                                                          | $\max(T_1, T_2, ..., T_N)$                                                                                                                                                                 |
| **4. min\_tokens**                | Số lượng **token nhỏ nhất** trong 1 đoạn văn bản                                                          | $\min(T_1, T_2, ..., T_N)$                                                                                                                                                                 |
| **5. subword\_rate**              | **Tỷ lệ token là subword** (từ bị tách ra thành các mảnh nhỏ) <br> ví dụ “học\_sinh” → \["học", "\_sinh"] | $\text{Subword Rate} = \frac{\text{Số token có dạng subword}}{\text{Tổng số token}}$ <br> Cách nhận dạng subword tùy tokenizer: <br> - `##` (BERT) <br> - không có `▁` đầu (SentencePiece) |
| **6. out\_of\_vocab\_rate (OOV)** | **Tỷ lệ token bị thay thế bằng mã không xác định** như `[UNK]`, `<unk>`                                   | $\text{OOV Rate} = \frac{\text{Số token là [UNK] hoặc <unk>}}{\text{Tổng số token}}$                                                                                                       |
| **7. time\_per\_1k\_samples**     | **Thời gian thực thi tokenization cho 1000 văn bản**                                                      | Đo bằng `time.time()` trước và sau khi tokenizer chạy trên 1000 mẫu: <br> $\text{Time} = \text{end\_time} - \text{start\_time}$                                                            |


In [135]:
def average_tokens_length(dataframe, tokenizer):
    """
    Tính toán trung bình số lượng tokens có trong một sample trên toàn bộ dataset.

    Args:
        - dataframe: Một dataframe có cột tên là "content_embedding", cột này chứa văn bản cần embedding.
        - tokenizer: Một đối tượng tokenizer (AutoTokenizer) dùng để tokenization văn bản.
    
    Returns:
        - list_tokens: Một 2D array, là toàn bộ tokens của dataset. Ex: [[tokens of a sample], [tokens of another sample]].
        - average_tokens: Trung bình số lượng tokens có trong một sample trên toàn bộ dataset
    """
    total_tokens = 0
    list_tokens = []
    for idx in range(len(dataframe)):
        sentence = dataframe["content_embedding"][idx]
        tokens = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        list_tokens.append(tokenizer.convert_ids_to_tokens(tokens['input_ids'][0]))
        total_tokens += len(tokenizer.convert_ids_to_tokens(tokens['input_ids'][0]))
    average_tokens = total_tokens / len(dataframe)
    return list_tokens, average_tokens

In [136]:
def std_tokens_length(list_tokens, average_tokens):
    """
    Tính toán độ lệch chuẩn số token giữa các sample trên toàn bộ dataset.

    Args:
        - list_tokens: 2D array, là toàn bộ tokens của dataset. Ex: [[tokens of a sample], [tokens of another sample]].
        - average_tokens: Trung bình lượng tokens trong một sample trên toàn bộ dataset.

    Returns:
        - Độ lệch chuẩn
    """
    M = 0
    for tokens in list_tokens:
        M += (len(tokens) - average_tokens) ** 2
    return (M / len(list_tokens)) ** 0.5

In [137]:
def min_max_token_length(list_tokens):
    """
    Tìm sample có số lượng tokens nhỏ nhất và lớn nhất.
    
    Args:
        - list_tokens: 2D array, là toàn bộ tokens của dataset. Ex: [[tokens of a sample], [tokens of another sample]]

    Returns:
        - Lượng tokenss ít nhất trong một sample.
        - Lượng tokens nhiều nhất trong một sample.
    """
    min_tokens = 0
    max_tokens = 0
    for tokens in list_tokens:
        if len(tokens) < min_tokens:
            min_tokens = len(tokens)
        if len(tokens) > max_tokens:
            max_tokens = len(tokens)

    return min_tokens, max_tokens

In [138]:
def subword_rate(list_tokens, subword_character):
    """
    Tính toán tỉ lệ token là subword trên toàn bộ dataset.
    
    Arg:
        - list_tokens: 2D array, là toàn bộ tokens của dataset. Ex: [[tokens of a sentences], [tokens of another sentences]]
        - subword_character: Một ký tự đánh dấu token đó là subword. Ex: ##, _, @@

    Returns:
        - Tỉ lệ tokens bị phân tách thành subword.
    """
    total_tokens = 0
    num_subword_tokens = 0
    for tokens in list_tokens:
        for token in tokens:
            if subword_character in token:
                num_subword_tokens += 1
        total_tokens += len(tokens)
    return num_subword_tokens / total_tokens

In [139]:
def out_of_vocab_rate(list_tokens, oov_character):
    """
    Tỉ lệ tokens bị thay thế bằng mã không xác định <UNK>/ <unk> (out-of-vocab rate) trên toàn bộ đataset.

    Args:
        - list_tokens: Một 2D array, là toàn bộ tokens của dataset. Ex: [[tokens of a sentences], [tokens of another sentences]]
        - oov_character (out-of-vocab): Ký tự giúp nhận viết đó là một token oov.
    """
    num_oov = 0
    total_tokens = 0
    for tokens in list_tokens:
        for token in tokens:
            if token == oov_character:
                num_oov += 1
        total_tokens += len(tokens)

    return num_oov / total_tokens
    

In [140]:
def all_categories(dataframe, tokenizer, subword_character, oov_character):
    """
    Gọi tất cả các hàm trên.
    """
    list_tokens, average_tokens = average_tokens_length(dataframe=dataframe, tokenizer=tokenizer)
    print(f"Average tokens length: {average_tokens}")

    std_tokens_ = std_tokens_length(list_tokens=list_tokens, average_tokens=average_tokens)
    print(f"Std of tokens length: {std_tokens_}")

    min_tokens, max_tokens = min_max_token_length(list_tokens=list_tokens)
    print(f"Minimum of tokens on a sample: {min_tokens}")
    print(f"Maximum of tokens on a sample: {max_tokens}")

    subword_rate_ = subword_rate(list_tokens=list_tokens, subword_character=subword_character)
    print(f"Subword rate: {subword_rate_}")

    oov_rate = out_of_vocab_rate(list_tokens=list_tokens,oov_character=oov_character)
    print(f"Out-of-vocab rate: {oov_rate}")

# all-MiniLM-L6-v2

In [146]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [147]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

512
3
{'input_ids': tensor([[  101, 14684,  5063,  1024, 20934, 10448,  1102,  4887,  5622,  2818,
         10514, 16371, 10085, 11937,   100,   100,  1006,   100,  1007,   100,
           100,   100,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [148]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 23])
tensor([[  101, 14684,  5063,  1024, 20934, 10448,  1102,  4887,  5622,  2818,
         10514, 16371, 10085, 11937,   100,   100,  1006,   100,  1007,   100,
           100,   100,   102]])

token_type_ids: torch.Size([1, 23])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 23])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

['[CLS]', 'chu', '##ong', ':', 'bu', '##oi', 'đ', '##au', 'li', '##ch', 'su', 'nu', '##oc', 'ta', '[UNK]', '[UNK]', '(', '[UNK]', ')', '[UNK]', '[UNK]', '[UNK]', '[SEP]']
chu -> 14684
##ong -> 5063
: -> 1024
bu -> 20934
##oi -> 10448
đ -> 1102
##au -> 4887
li -> 5622
##ch -> 2818
su -> 10514
nu -> 16371
##oc -> 10085
ta -> 11937
[UNK] -> 100
[UNK] -> 100
( -> 1006
[UNK] -> 100
) -> 1007
[UNK] -> 100
[UNK] -> 100
[UNK] -> 100


In [150]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="##", oov_character="[UNK]")

Average tokens length: 332.71097234611955
Std of tokens length: 195.5535530481586
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 512
Subword rate: 0.35267810461459265
Out-of-vocab rate: 0.0


# bert-base-vietnamese-uncased

In [151]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/bert-base-vietnamese-uncased"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [152]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1000000000000000019884624838656
3
{'input_ids': tensor([[    2, 22360,     1,  3710,   374,    27,   881, 15780,  1460,  3086,
          3332,   389,     1,     1,     1,     1,     1,     1,     1,     1,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [153]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 21])
tensor([[    2, 22360,     1,  3710,   374,    27,   881, 15780,  1460,  3086,
          3332,   389,     1,     1,     1,     1,     1,     1,     1,     1,
             3]])

token_type_ids: torch.Size([1, 21])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 21])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

['[CLS]', 'chuong', '[UNK]', 'bu', '##o', '##i', 'đau', 'lich', 'su', 'nu', '##oc', 'ta', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[SEP]']
chuong -> 22360
[UNK] -> 1
bu -> 3710
##o -> 374
##i -> 27
đau -> 881
lich -> 15780
su -> 1460
nu -> 3086
##oc -> 3332
ta -> 389
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1
[UNK] -> 1


In [154]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="##", oov_character="[UNK]")

Average tokens length: 402.99107939339876
Std of tokens length: 354.1514993442404
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 2240
Subword rate: 0.16576979012867651
Out-of-vocab rate: 0.11234900487655865


# distiluse-base-multilingual-cased-v2

In [155]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/distiluse-base-multilingual-cased-v2"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [156]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

512
2
{'input_ids': tensor([[   101,  88797,    131,    100,    100,    100,    100,    100,  91075,
            100,    100,    113,    100,    114,    400, 111756,   2673,    100,
            102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [157]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 19])
tensor([[   101,  88797,    131,    100,    100,    100,    100,    100,  91075,
            100,    100,    113,    100,    114,    400, 111756,   2673,    100,
            102]])

attention_mask: torch.Size([1, 19])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

['[CLS]', 'Chương', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'TA', '[UNK]', '[UNK]', '(', '[UNK]', ')', 'ʃ', '##♪', '卍', '[UNK]', '[SEP]']
Chương -> 88797
: -> 131
[UNK] -> 100
[UNK] -> 100
[UNK] -> 100
[UNK] -> 100
[UNK] -> 100
TA -> 91075
[UNK] -> 100
[UNK] -> 100
( -> 113
[UNK] -> 100
) -> 114
ʃ -> 400
##♪ -> 111756
卍 -> 2673
[UNK] -> 100


In [158]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="##", oov_character="[UNK]")

Average tokens length: 295.4228367528992
Std of tokens length: 188.4914893252836
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 512
Subword rate: 0.11856786112226687
Out-of-vocab rate: 0.016734658135272324


# multilingual-e5-small

In [159]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [160]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

512
2
{'input_ids': tensor([[     0,  40691,     12,  22013,  99973,    568,   4428, 240532,   1062,
            339, 119393,  16999,    159, 160527,    541, 227860,  12998,      6,
         247723,      6,  61893,      3,    132, 244806, 245728, 244806,     16,
         245313,  26547,      6, 246342,      6,      3,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [161]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 34])
tensor([[     0,  40691,     12,  22013,  99973,    568,   4428, 240532,   1062,
            339, 119393,  16999,    159, 160527,    541, 227860,  12998,      6,
         247723,      6,  61893,      3,    132, 244806, 245728, 244806,     16,
         245313,  26547,      6, 246342,      6,      3,      2]])

attention_mask: torch.Size([1, 34])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

['<s>', '▁Chương', ':', '▁BU', 'Ổ', 'I', '▁Đ', 'Ầ', 'U', '▁L', 'Ị', 'CH', '▁S', 'Ử', '▁N', 'ƯỚC', '▁TA', '▁', '❉', '▁', '♡', '<unk>', '(', 'ˆ', '◡', 'ˆ', ')', 'ʃ', '♪', '▁', '卍', '▁', '<unk>', '</s>']
▁Chương -> 40691
: -> 12
▁BU -> 22013
Ổ -> 99973
I -> 568
▁Đ -> 4428
Ầ -> 240532
U -> 1062
▁L -> 339
Ị -> 119393
CH -> 16999
▁S -> 159
Ử -> 160527
▁N -> 541
ƯỚC -> 227860
▁TA -> 12998
▁ -> 6
❉ -> 247723
▁ -> 6
♡ -> 61893
ƪ -> 3
( -> 132
ˆ -> 244806
◡ -> 245728
ˆ -> 244806
) -> 16
ʃ -> 245313
♪ -> 

In [ ]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="▁", oov_character="<unk>")

Average tokens length: 293.45673505798396
Std of tokens length: 185.51125237072657
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 512
Subword rate: 0.77908288115757
Out-of-vocab rate: 0.0


# phobert-base-v2

In [167]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

Some weights of RobertaModel were not initialized from the model checkpoint at ../../embedding_models/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1000000000000000019884624838656
3
{'input_ids': tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [169]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 31])
tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]])

token_type_ids: torch.Size([1, 31])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 31])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

['<s>', 'Ch@@', 'ương@@', ':', 'B@@', 'U@@', 'ỔI', 'Đ@@', 'Ầ@@', 'U', 'L@@', 'Ị@@', 'CH', 'S@@', 'Ử', 'N@@', 'ƯỚC', 'TA', '<unk>', '<unk>', '<unk>', '(@@', '<unk>', '<unk>', '<unk>', ')@@', 'ʃ@@', '<unk>', '<unk>', '<unk>', '</s>']
Ch@@ -> 1735
ương@@ -> 50410
: -> 27
B@@ -> 924
U@@ -> 1878
ỔI -> 59831
Đ@@ -> 1782
Ầ@@ -> 28661
U -> 3521
L@@ -> 1042
Ị@@ -> 31584
CH -> 6506
S@@ -> 870
Ử -> 55132
N@@ -> 1086
ƯỚC 

In [170]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="@@", oov_character="<unk>")

Average tokens length: 368.921498661909
Std of tokens length: 313.0481961962608
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 1932
Subword rate: 0.2056794523661564
Out-of-vocab rate: 0.008985373379017848


# phobert-large

In [171]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/phobert-large"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [172]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1000000000000000019884624838656
3
{'input_ids': tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [173]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 31])
tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]])

token_type_ids: torch.Size([1, 31])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 31])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

['<s>', 'Ch@@', 'ương@@', ':', 'B@@', 'U@@', 'ỔI', 'Đ@@', 'Ầ@@', 'U', 'L@@', 'Ị@@', 'CH', 'S@@', 'Ử', 'N@@', 'ƯỚC', 'TA', '<unk>', '<unk>', '<unk>', '(@@', '<unk>', '<unk>', '<unk>', ')@@', 'ʃ@@', '<unk>', '<unk>', '<unk>', '</s>']
Ch@@ -> 1735
ương@@ -> 50410
: -> 27
B@@ -> 924
U@@ -> 1878
ỔI -> 59831
Đ@@ -> 1782
Ầ@@ -> 28661
U -> 3521
L@@ -> 1042
Ị@@ -> 31584
CH -> 6506
S@@ -> 870
Ử -> 55132
N@@ -> 1086
ƯỚC 

In [174]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="@@", oov_character="<unk>")

Average tokens length: 368.921498661909
Std of tokens length: 313.0481961962608
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 1932
Subword rate: 0.2056794523661564
Out-of-vocab rate: 0.008985373379017848


# sup-SimCSE-VietNamese-phobert-base

In [175]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/sup-SimCSE-VietNamese-phobert-base"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [176]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

256
3
{'input_ids': tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [177]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 31])
tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]])

token_type_ids: torch.Size([1, 31])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 31])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

['<s>', 'Ch@@', 'ương@@', ':', 'B@@', 'U@@', 'ỔI', 'Đ@@', 'Ầ@@', 'U', 'L@@', 'Ị@@', 'CH', 'S@@', 'Ử', 'N@@', 'ƯỚC', 'TA', '<unk>', '<unk>', '<unk>', '(@@', '<unk>', '<unk>', '<unk>', ')@@', 'ʃ@@', '<unk>', '<unk>', '<unk>', '</s>']
Ch@@ -> 1735
ương@@ -> 50410
: -> 27
B@@ -> 924
U@@ -> 1878
ỔI -> 59831
Đ@@ -> 1782
Ầ@@ -> 28661
U -> 3521
L@@ -> 1042
Ị@@ -> 31584
CH -> 6506
S@@ -> 870
Ử -> 55132
N@@ -> 1086
ƯỚC 

In [178]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="@@", oov_character="<unk>")

Average tokens length: 188.19446922390722
Std of tokens length: 86.92475036676232
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 256
Subword rate: 0.22599850212830502
Out-of-vocab rate: 0.00996843093199852


# vietnamese-bi-encoder

In [179]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/vietnamese-bi-encoder"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [180]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1000000000000000019884624838656
3
{'input_ids': tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [181]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 31])
tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]])

token_type_ids: torch.Size([1, 31])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 31])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

['<s>', 'Ch@@', 'ương@@', ':', 'B@@', 'U@@', 'ỔI', 'Đ@@', 'Ầ@@', 'U', 'L@@', 'Ị@@', 'CH', 'S@@', 'Ử', 'N@@', 'ƯỚC', 'TA', '<unk>', '<unk>', '<unk>', '(@@', '<unk>', '<unk>', '<unk>', ')@@', 'ʃ@@', '<unk>', '<unk>', '<unk>', '</s>']
Ch@@ -> 1735
ương@@ -> 50410
: -> 27
B@@ -> 924
U@@ -> 1878
ỔI -> 59831
Đ@@ -> 1782
Ầ@@ -> 28661
U -> 3521
L@@ -> 1042
Ị@@ -> 31584
CH -> 6506
S@@ -> 870
Ử -> 55132
N@@ -> 1086
ƯỚC 

In [182]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="@@", oov_character="<unk>")

Average tokens length: 368.921498661909
Std of tokens length: 313.0481961962608
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 1932
Subword rate: 0.2056794523661564
Out-of-vocab rate: 0.008985373379017848


# vietnamese-embedding

In [183]:
from transformers import AutoTokenizer
from transformers import AutoModel

path = "../../embedding_models/vietnamese-embedding"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path)

In [184]:
print(tokenizer.model_max_length)

# Tokenizer cho câu đầu vào
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
print(len(inputs))
print(inputs)

512
3
{'input_ids': tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [185]:
# Show inputs
for k, v in inputs.items():
    print(f"{k}: {v.shape}")
    print(v)
    print()

# Hiển thị chunking
print("==========================")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

# Hiển thị token và mã tương ứng
print("==========================")
for token, token_id in zip(tokenizer.tokenize(sentence), tokenizer(sentence)["input_ids"][1:-1]):
    print(f"{token} -> {token_id}") 

input_ids: torch.Size([1, 31])
tensor([[    0,  1735, 50410,    27,   924,  1878, 59831,  1782, 28661,  3521,
          1042, 31584,  6506,   870, 55132,  1086, 55296, 10875,     3,     3,
             3, 14157,     3,     3,     3, 37272, 63120,     3,     3,     3,
             2]])

token_type_ids: torch.Size([1, 31])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]])

attention_mask: torch.Size([1, 31])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

['<s>', 'Ch@@', 'ương@@', ':', 'B@@', 'U@@', 'ỔI', 'Đ@@', 'Ầ@@', 'U', 'L@@', 'Ị@@', 'CH', 'S@@', 'Ử', 'N@@', 'ƯỚC', 'TA', '<unk>', '<unk>', '<unk>', '(@@', '<unk>', '<unk>', '<unk>', ')@@', 'ʃ@@', '<unk>', '<unk>', '<unk>', '</s>']
Ch@@ -> 1735
ương@@ -> 50410
: -> 27
B@@ -> 924
U@@ -> 1878
ỔI -> 59831
Đ@@ -> 1782
Ầ@@ -> 28661
U -> 3521
L@@ -> 1042
Ị@@ -> 31584
CH -> 6506
S@@ -> 870
Ử -> 55132
N@@ -> 1086
ƯỚC 

In [186]:
all_categories(dataframe=df, tokenizer=tokenizer, subword_character="@@", oov_character="<unk>")

Average tokens length: 293.11329170383584
Std of tokens length: 185.41635433026227
Minimum of tokens on a sample: 0
Maximum of tokens on a sample: 512
Subword rate: 0.21116318704729442
Out-of-vocab rate: 0.00945888368129527


# Summary

Để so sánh **hiệu quả tokenization** giữa các tokenizer/model khác nhau, có thể dùng một số **chỉ số định lượng và định tính** :


### 🧮 **1. Chỉ số định lượng (Quantitative Metrics)**

#### ✅ **a. Trung bình số tokens / mẫu**

* **Mục đích**: Đánh giá độ “nén” của tokenizer (ít tokens hơn cho cùng nội dung → tốt hơn).
* **Công thức**:

  $$
  \text{Avg. tokens per sample} = \frac{\sum_{i=1}^{N} \text{len(tokens}_i\text{)}}{N}
  $$

#### ✅ **b. Tỷ lệ “subword splitting”**

* **Mục đích**: Tokenizer tốt sẽ ít phải cắt từ tiếng Việt thành nhiều mảnh.
* **Công thức**:

  $$
  \text{Subword ratio} = \frac{\text{Tổng số subwords}}{\text{Tổng số từ gốc}}
  $$

#### ✅ **c. Vocabulary coverage (nếu có từ điển chuẩn)**

* **Mục đích**: Đo mức độ bao phủ từ vựng tiếng Việt thực tế.
* So với từ điển chuẩn hoặc tập wordlist corpus.
* **Chỉ số**:

  * % từ vựng có trong vocab
  * % từ vựng bị tách thành subwords

#### ✅ **d. Tỷ lệ token đặc biệt / padding**

* Xem tokenizer có sinh nhiều `[PAD]`, `[UNK]` không (dấu hiệu thiếu hiệu quả).


### 📊 **2. Chỉ số định tính (Qualitative Analysis)**

#### 🔍 **a. Các ví dụ token hóa thực tế**

* So sánh cách tokenizer xử lý các câu tiếng Việt có:

  * từ ghép
  * dấu tiếng Việt (dấu hỏi, ngã, sắc…)
  * từ mượn, từ chuyên ngành
  * câu ngắn/dài, chính tả không chuẩn

#### 🎯 **b. Độ nguyên vẹn từ vựng**

* Tokenizer tốt nên giữ được từ đơn/từ ghép tiếng Việt như `"học sinh"`, `"giáo viên"` thay vì `"học", "##sinh"`


### ⚙️ **3. Thực nghiệm phụ trợ (nếu cần mở rộng)**

* Thử huấn luyện nhanh 1 downstream task như classification (sentiment, intent, etc.) với từng tokenizer → so sánh F1/accuracy để xem ảnh hưởng thực tế.
* Đo thời gian tokenize + memory usage nếu đánh giá hiệu suất.


### 📑 Gợi ý trình bày báo cáo

| Mô hình   | Avg Tokens | Subword Ratio | % OOV | Ví dụ token hóa                        |
| --------- | ---------- | ------------- | ----- | -------------------------------------- |
| BERT Pho  | 54.3       | 1.21          | 2.3%  | "giáo viên" → `['giáo', 'viên']`       |
| GPT2-ViT5 | 48.6       | 1.09          | 1.1%  | "giáo viên" → `['giáo viên']`          |
| mBERT     | 61.2       | 1.35          | 3.7%  | "giáo viên" → `['gi', '##áo', 'viên']` |
